In [1]:
from xbpy import rdutil
import pymolviz as pmv
import numpy as np

In [2]:
results_path = "/mnt/c/Users/highgarden/Docker/DiffDock/data/5ckr_docking/full_others/"
base_ligand_path = "/mnt/c/Users/highgarden/Docker/DiffDock/data/5ckr_docking/ligands/"
cur_ligand = "Mureidomycin_A_PDB_kekulized"

In [1]:
from rdkit import Chem

In [11]:
Chem.MolFromSmarts("[OX1]~*~[OX1]")

In [3]:
aligned_mol = next(rdutil.read_molecules(base_ligand_path + cur_ligand + ".sdf"))
mol = next(rdutil.read_molecules(results_path + cur_ligand + "/*.sdf"))

In [4]:
common_substructure = next(rdutil.read_molecules("uridyl_group.pdb"))

In [5]:
h_add_mol = rdutil.add_explicit_hydrogens(mol)
rdutil.write_molecules(h_add_mol, "aligned_mol.sdf")

In [5]:
from rdkit import Chem
from rdkit.Chem.rdFMCS import FindMCS

In [6]:
# Get Indices of common substructure
substruct_mol = Chem.RWMol(mol)
substruct_aligned_mol = Chem.RWMol(aligned_mol)
for bond in substruct_mol.GetBonds():
    bond.SetBondType(Chem.BondType.UNSPECIFIED)
for bond in substruct_aligned_mol.GetBonds():
    bond.SetBondType(Chem.BondType.UNSPECIFIED)
common_substructure = Chem.RWMol(common_substructure)
for bond in common_substructure.GetBonds():
    bond.SetBondType(Chem.BondType.UNSPECIFIED)
MCS = FindMCS([substruct_mol, common_substructure]).queryMol
common_substructure_indices = substruct_mol.GetSubstructMatches(MCS)
aligned_substructure_indices = substruct_aligned_mol.GetSubstructMatches(MCS)

In [11]:
max_depth = 2

In [12]:
# write substructures
matched_mol = rdutil.keep_atoms(substruct_mol, common_substructure_indices[0])
matched_aligned_mol = rdutil.keep_atoms(substruct_aligned_mol, aligned_substructure_indices[0])
rdutil.write_molecules(substruct_mol, "substruct_mol.sdf")
rdutil.write_molecules(matched_mol, "matched_mol.sdf")
rdutil.write_molecules(matched_aligned_mol, "matched_aligned_mol.sdf")

In [13]:
aligned_positions = rdutil.seed_align(aligned_mol, mol, aligned_substructure_indices[0], common_substructure_indices[0], up_to_depth=max_depth)

ColorMap.py:180 Infered color [0.76321056 0.58384606 0.19465687] and alpha 1 from value (0.7632105624545802, 0.5838460616396939, 0.19465686802007026)
[[5, 13], [5, 16], [8, 27], [11, 31], [14, 37], [17, 37]]
[[5, 13], [5, 16], [8, 27], [11, 31], [14, 37], [17, 37]]
Handling bond (1, 4) at depth 0
Handling bond (0, 2) at depth 1
Handling bond (25, 8) at depth 1
Handling bond (27, 29) at depth 2
Handling bond (29, 11) at depth 3
Handling bond (31, 33) at depth 4
Handling bond (33, 14) at depth 5
Handling bond (17, 38) at depth 6
Handling bond (38, 46) at depth 7
Handling bond (38, 39) at depth 7
Handling bond (39, 40) at depth 8
Handling bond (44, 15) at depth 9
Handling bond (33, 34) at depth 5
Handling bond (34, 35) at depth 6
Handling bond (35, 3) at depth 7
Handling bond (3, 36) at depth 8
Handling bond (29, 47) at depth 3
Handling bond (47, 20) at depth 4
Handling bond (20, 49) at depth 5
Handling bond (49, 50) at depth 6
Handling bond (50, 23) at depth 7
Handling bond (50, 51) at d

In [14]:
# apply positions to aligned_mol
m = Chem.RWMol(aligned_mol)
for i in range(len(aligned_positions)):
    m.GetConformer().SetAtomPosition(i, aligned_positions[i])

rdutil.write_molecules(m, f"test_aligned_uridyl_{max_depth}.pdb")